export dataset: to `ic2s2_datathon`

update `helpers.extract_user_text_data`:
    - include who posted the `text`
    - include pre-computed politeness score

---

In [1]:
import os, helpers, warnings, tqdm
warnings.simplefilter('once')

In [2]:
import pandas as pd

In [3]:
from sqlalchemy import create_engine
engine = create_engine('postgresql://zhiyzuo@localhost:5432/ic2s2_datathon')

/Users/zhiyzuo/.virtualenvs/Zuo-Py2/lib/python2.7/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


---

loop through all excel files

In [4]:
base_dir = 'excel-data/'

In [5]:
file_list = [base_dir+f for f in os.listdir(base_dir) if f.endswith('.xlsx')]
len(file_list)

85

use a for loop to create a giant user and text file

In [6]:
user_df = list()
text_df = list()

In [7]:
file_list[38]

'excel-data/score_Cervical Cancer.xlsx'

In [8]:
for f in tqdm.tqdm_notebook(file_list):
    #print(f)
    u_df, t_df = helpers.extract_user_text_data(f)
    user_df.append(u_df)
    text_df.append(t_df)
len(user_df) == len(text_df), len(text_df) == len(file_list)

/Users/zhiyzuo/.virtualenvs/Zuo-Py2/lib/python2.7/site-packages/pandas/core/indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
helpers.py:63: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  question_df['type'] = 'q'


(True, True)

In [9]:
user_df = pd.concat(user_df,ignore_index=True)
user_df.shape

(21995, 5)

In [10]:
user_df.head(2)

,age,gender,role,user_id,id
0,74.0,Female,Nurse,"Faye Lang, RN, MSW",0
1,74.0,Female,Nurse,"Faye Lang, RN, MSW",1


duplicates

In [11]:
user_df.drop_duplicates(subset=['user_id'], inplace=True, keep='first')
user_df.shape

(9005, 5)

In [12]:
user_df.user_id.isnull().sum()

0

In [13]:
text_df = pd.concat(text_df, ignore_index=True)
text_df.shape

(22003, 8)

In [14]:
text_df.head(2)

,qa_id,text,asker_id,answerer_id,politeness,type,post_user_id,disease
0,0,"Hello, I'm a male in his 20s and I've had fati...",TigerShot,"Faye Lang, RN, MSW",0.75,q,TigerShot,psychiatric topics
1,1,Hello.. You might remember me contacting you 3...,marcia smith,"Faye Lang, RN, MSW",0.66,q,marcia smith,psychiatric topics


In [15]:
text_df.post_user_id.isnull().sum()

0

save to a database

In [16]:
user_df.to_sql('user', con=engine, index=False, if_exists='replace')
text_df.to_sql('text', con=engine, index=False, if_exists='replace')